#  Анализ сервиса для чтения книг по подписке

Цель данного исследования - проанализировать базу данных купленного нашей компанией сервиса для чтения книг по подписке.  
Основыне задачи:
- [посчитать сколько книг вышло после 1 января 2000 года;](#task_1)
- [для каждой книги посчитать количество обзоров и среднюю оценку;](#task_2)
- [определить издательство, которое выпустило наибольшее число книг толще 50 страниц, чтобы исключить из анализа брошюры;](#task_3)
- [определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;](#task_4)
- [посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.](#task_5)

## Загрузка данных и подготовка к анализу

### Загрузка библиотек

Импротируем библиотеки

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

Устанавливаем параметры подключения.

In [2]:
# устанавливаем параметры
db_config = {'user': 'username', # имя пользователя
'pwd': 'pass', # пароль
'host': 'host.net',
'port': 6432, # порт подключения
'db': 'db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


### Исследование данных

Загрузим по 5 строк каждой таблицы и изучим их, а также посчитаем количество строк в таблицах.

**Таблица `books`**

In [3]:
query = '''
    SELECT * 
    FROM books 
    LIMIT 5
    '''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''
    SELECT COUNT(*) 
    FROM books 
    '''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,1000


Описание данных:  
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.  

6 столбцов, 1000 строк.

**Таблица `authors`**

In [5]:
query = '''
    SELECT * 
    FROM authors 
    LIMIT 5
    '''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = '''
    SELECT COUNT(*) 
    FROM authors 
    '''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,636


Описание данных:
- `author_id` — идентификатор автора;
- `author` — имя автора.  
  
2 столбца, 636 строк.

**Таблица `publishers`**

In [7]:
query = '''
    SELECT * 
    FROM publishers 
    LIMIT 5
    '''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
query = '''
    SELECT COUNT(*) 
    FROM publishers 
    '''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,340


Описание данных:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства.
  
2 столбца, 340 строк.

**Таблица `ratings`**

In [9]:
query = '''
    SELECT * 
    FROM ratings 
    LIMIT 5
    '''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
query = '''
    SELECT COUNT(*) 
    FROM ratings 
    '''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,6456


Описание данных:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.  
  
4 столбца, 6456 строк.

**Таблица `reviews`**

In [11]:
query = '''
    SELECT * 
    FROM reviews 
    LIMIT 5
    '''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [12]:
query = '''
    SELECT COUNT(*) 
    FROM reviews 
    '''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,2793


Описание данных:
- `review_id`— идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.
  
4 столбца, 2793 строк.

В нашем распоряжении 5 таблиц:
- books - 6 столбцов, 1000 строк,
- authors - 2 столбца, 636 строк.
- publishers - 2 столбца, 340 строк,
- ratings - 4 столбца, 6456 строк,
- reviews - 4 столбца, 2793 строк.

Схема данных:  
![](https://pictures.s3.yandex.net/resources/scheme_1589269096.png)


## Задача 1 

Посчитаем, сколько книг вышло после 1 января 2000 года. <a id='task_1'></a>

In [13]:
query = '''
    SELECT COUNT(book_id) 
    FROM books 
    WHERE publication_date > '2000-01-01'
    '''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


С 1 января 2000 года вышло 819 книг.

## Задача 2

Для каждой книги посчитаем количество обзоров и среднюю оценку. <a id='task_2'></a>

In [14]:
query = '''
    SELECT b.title,
        COUNT(DISTINCT review_id) AS reviews,
        AVG(rat.rating) AS avg_rating
    FROM books AS b
    LEFT JOIN reviews AS rev ON b.book_id =  rev.book_id
    LEFT JOIN ratings AS rat ON b.book_id =  rat.book_id
    GROUP BY b.book_id, b.title
    ORDER BY reviews DESC, avg_rating DESC
    '''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,reviews,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Book Thief,6,4.264151
4,The Glass Castle,6,4.206897
...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
996,Leonardo's Notebooks,0,4.000000
997,Essential Tales and Poems,0,4.000000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


## Задача 3

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц, чтобы исключить из анализа брошюры. <a id='task_3'></a>

In [15]:
query = '''
    SELECT p.publisher,
        COUNT(b.book_id) AS books
    FROM books AS b
    LEFT JOIN publishers AS p ON b.publisher_id=p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY b.publisher_id, p.publisher
    ORDER BY books DESC
    LIMIT 1
    '''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,books
0,Penguin Books,42


Издательство Penguin Books выпустило 42 книги более 50 страниц.

## Задача 4 

Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками. <a id='task_4'></a>

In [16]:
query = '''
    SELECT a.author,
            AVG(r.rating) AS avg_rating
    FROM books AS b
    LEFT JOIN ratings AS r ON b.book_id =  r.book_id
    LEFT JOIN authors AS a ON b.author_id =  a.author_id
    WHERE b.book_id IN (
        SELECT book_id
        FROM ratings
        GROUP BY book_id
        HAVING COUNT(rating_id) >= 50)
    GROUP BY a.author_id, a.author
    ORDER BY avg_rating DESC
    LIMIT 1
    '''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Автором с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками, является Джоан Роулинг, совместно с Мэри ГрандПре.

## Задача 5

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок. <a id='task_5'></a>

In [17]:
query = '''
    SELECT AVG(COUNT(review_id)) OVER () AS avg_rev_count
    FROM reviews
    WHERE username in (
        SELECT username 
        FROM  ratings
        GROUP BY username
        HAVING COUNT(rating_id) > 48)
    GROUP BY username
    LIMIT 1
    '''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_rev_count
0,24.0


Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24.

## Вывод

Мы проанализировали базу данных купленного нашей компанией сервиса для чтения книг по подписке:
- с 1 января 2000 года вышло 819 книг;
- для каждой книги посчитали количество обзоров и среднюю оценку и вывели на экран;
- определили издательство, которое выпустило наибольшее число книг толще 50 страниц, им оказалось Penguin Books - оно выпустило 42 книги;
- определили автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками - им оказалась Джоан Роулинг, совместно с Мэри ГрандПре;
- посчитали среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24 обзора.

## Рекомендации

Рекомендуем обратить внимание на книги Джоан Роулинг, как самые высокооцененные, а также на издательство Penguin Books, как посталяющее больше всего книг, для наполнения сервиса.  